# 训练Xception模型，并保存权重

1 使用 Xception

2 冻结训练分类器

3 finetune训练整个网络

### 1 import libs

In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


### 2 build pretrained model

In [2]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
base_model.summary()
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(4, activation='softmax')(p_out)

for layer in base_model.layers:  
     layer.trainable = False

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

### 3 build param

In [3]:
batch_size = 64
epochs = 1

total_imgs_num = 79309 + 3491
train_imgs_num = 79309
valid_imgs_num = 3491

img_size = (299, 299)

### 4 build data & train

In [4]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("../oct-dataset/OCT2017/train-ready/", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../oct-dataset/OCT2017/valid-ready/", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("Xception_first_train.h5")

Found 79309 images belonging to 4 classes.
Found 4175 images belonging to 4 classes.
Epoch 1/1
1240/1239 [==============================] - 482s 388ms/step - loss: 0.7267 - acc: 0.7344 - val_loss: 0.5539 - val_acc: 0.8086


### 5 Finetune

In [2]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(4, activation='softmax')(p_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception_first_train.h5')
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [3]:
batch_size = 24
epochs = 10

total_imgs_num = 79309 + 3491
train_imgs_num = 79309
valid_imgs_num = 3491

img_size = (299, 299)

img_gen_t = ImageDataGenerator(#rotation_range=15,
                               #width_shift_range=0.3,
                               #height_shift_range=0.3,
                               #zoom_range=0.1,
                               #horizontal_flip=True,
                               #vertical_flip=True)
)
train_generator = img_gen_t.flow_from_directory("../oct-dataset/OCT2017/train-ready/", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../oct-dataset/OCT2017/valid-ready/", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

gen = ImageDataGenerator()

model.save_weights("Xception_finetune.h5")

Found 79309 images belonging to 4 classes.
Found 4175 images belonging to 4 classes.
Epoch 1/10
3305/3304 [==============================] - 1494s 452ms/step - loss: 0.1552 - acc: 0.9493 - val_loss: 0.1536 - val_acc: 0.9526
Epoch 2/10
3305/3304 [==============================] - 1400s 423ms/step - loss: 0.0754 - acc: 0.9751 - val_loss: 0.1179 - val_acc: 0.9701
Epoch 3/10
3305/3304 [==============================] - 1400s 423ms/step - loss: 0.0498 - acc: 0.9835 - val_loss: 0.1080 - val_acc: 0.9737
Epoch 4/10
3305/3304 [==============================] - 1400s 424ms/step - loss: 0.0333 - acc: 0.9893 - val_loss: 0.1040 - val_acc: 0.9727
Epoch 5/10
3305/3304 [==============================] - 1400s 424ms/step - loss: 0.0238 - acc: 0.9920 - val_loss: 0.1042 - val_acc: 0.9710
Epoch 6/10
3305/3304 [==============================] - 1400s 423ms/step - loss: 0.0178 - acc: 0.9935 - val_loss: 0.1400 - val_acc: 0.9732
Epoch 7/10
3305/3304 [==============================] - 1400s 424ms/step - loss: 

In [10]:
batch_size = 20
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("../oct-dataset/OCT2017/test-ready/", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
test = model.predict_generator(test_generator, (test_img_nums//batch_size) + 1, verbose=1)
test = test[:test_img_nums]
label = test_generator.classes

Found 1000 images belonging to 4 classes.
50/50 [==============================] - 5s 107ms/step


In [11]:
print(test.shape)
print(label.shape)

(1000, 4)
(1000,)


In [13]:
print(len(test))
print(len(label))

print(test[0])
print(np.argmax(test[0]))

true_nums = 0
total_nums = len(label)
for i in range(len(label)):
    if(label[i] == np.argmax(test[i])):
        true_nums = true_nums + 1

print(true_nums)
print(total_nums)
print(true_nums / total_nums)

1000
1000
[1.0000000e+00 4.2962565e-16 5.3597833e-17 8.1331025e-17]
0
994
1000
0.994


### 5 load weights & predict

In [43]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [44]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

Found 14 images belonging to 3 classes.
0
1
[array([ 0.30176267,  0.35524741,  0.34298989], dtype=float32), array([ 0.28475785,  0.34978592,  0.36545616], dtype=float32), array([ 0.3018229 ,  0.35525236,  0.34292471], dtype=float32), array([ 0.29986629,  0.35433695,  0.34579682], dtype=float32)]


In [45]:
print(np.array(predictions))

[[ 0.30176267  0.35524741  0.34298989]
 [ 0.28475785  0.34978592  0.36545616]
 [ 0.3018229   0.35525236  0.34292471]
 [ 0.29986629  0.35433695  0.34579682]]


### 6 write predictions 2 CSV